In [8]:
from sknetwork.ranking import PageRank
import pickle
from causallearn.search.FCMBased import lingam
from collections import defaultdict
import pandas as pd
import time
import numpy as np
from sklearn.preprocessing import normalize

In [10]:
hts_res_dict = {}
for experiment in range(0,4):
    with open(f"../data/fault_data/hts_fault_{experiment}.pkl", "rb") as f:
        s_list, X = pickle.load(f)
    ind = 0
    mapping = {}
    for service in s_list:
        x = service.split('-')
        x = x[:-2]
        x = '-'.join(x)
        mapping[ind] = x
        ind+=1
    # Start CausalRCA
    start_time = time.time()
    X = np.diff(X,axis=0)
    X = normalize(X,axis=0)
    model = lingam.DirectLiNGAM()
    model.fit(X)
    adj = model.adjacency_matrix_.T 
    pagerank = PageRank()
    scores = pagerank.fit_predict(adj.T)
    combined_pagerank = defaultdict(float)
    for service_id in range(len(scores)):
        combined_pagerank[mapping[service_id]] += scores[service_id]
    
    combined_pagerank_df = pd.DataFrame(
                list(combined_pagerank.items()), columns=["Service Type", "Combined Frequency"]
    )
    end_time = time.time()
    ####
    
    t_elapsed = end_time - start_time
    sorted_pr = combined_pagerank_df.sort_values(by="Combined Frequency", ascending=False)
    top_5 = list(sorted_pr.head(5)["Service Type"])
    hts_res_dict[experiment] = (sorted_pr, t_elapsed)

In [11]:
cp_res_dict = {}
for experiment in range(0,4):
    with open(f"../data/fault_data/cp_fault_{experiment}.pkl", "rb") as f:
        s_list, X = pickle.load(f)
    ind = 0
    mapping = {}
    for service in s_list:
        x = service.split('-')
        x = x[:-2]
        x = '-'.join(x)
        mapping[ind] = x
        ind+=1
    # Start CausalRCA
    start_time = time.time()
    X = np.diff(X,axis=0)
    X = normalize(X,axis=0)
    model = lingam.DirectLiNGAM()
    model.fit(X)
    adj = model.adjacency_matrix_.T 
    pagerank = PageRank()
    scores = pagerank.fit_predict(adj.T)
    combined_pagerank = defaultdict(float)
    for service_id in range(len(scores)):
        combined_pagerank[mapping[service_id]] += scores[service_id]
    
    combined_pagerank_df = pd.DataFrame(
                list(combined_pagerank.items()), columns=["Service Type", "Combined Frequency"]
    )
    end_time = time.time()
    ####
    
    t_elapsed = end_time - start_time
    sorted_pr = combined_pagerank_df.sort_values(by="Combined Frequency", ascending=False)
    top_5 = list(sorted_pr.head(5)["Service Type"])
    cp_res_dict[experiment] = (sorted_pr, t_elapsed)